In [ ]:
## import the necessary libraries
from thrift.transport import TTransport, TSocket
from thrift.protocol.TBinaryProtocol import TBinaryProtocol

from pyseek import SeekRPC
from pyseek.ttypes import SeekQuery, QueryParams, QueryResult

In [ ]:
## establish a server connection
host = 'localhost'
port = 9090

socket = TSocket.TSocket(host, port)
transport = TTransport.TBufferedTransport(socket)
protocol = TBinaryProtocol(transport)
client = SeekRPC.Client(protocol)
transport.open()

In [ ]:
## run a query
def runQuery(species, genes, useSymbols=False):
    params = QueryParams(distance_measure="ZscoreHubbinessCorrected",
                         min_query_genes_fraction=0.5,
                         min_genome_fraction=0.5,
                         use_gene_symbols=useSymbols)

    query = SeekQuery(species=species, genes=genes, parameters=params)
    
    result = client.seek_query(query)
    if result.success is True:
        for i, gs in enumerate(result.gene_scores):
            print(f'gene: {gs.name}, {gs.value}')
            if i > 100: break

        for i, ds in enumerate(result.dataset_weights):
            print(f'dset: {ds.name}, {ds.value}')
            if i > 100: break
    else:
        print(f'query error: {result.statusMsg}')


In [ ]:
species = 'fly'
genes = ['CG18094', 'CG10189']
# genes = [35234', '35232']
runQuery(species, genes, useSymbols=True)

In [ ]:
species = 'mouse'
genes = ['GLI1', 'GLI2', 'PTCH1']
# genes = ['14632', '14633', '19206']
runQuery(species, genes, useSymbols=True)

In [ ]:
species = 'human'
genes = ['SMO', 'PTCH1', 'PTCH2', 'BOC']
runQuery(species, genes, useSymbols=True)

In [ ]:
species = 'mock'
genes = ['90634', '23659']
runQuery(species, genes, useSymbols=False)

In [ ]:
## close the server connection
transport.close()